In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_code(symbol):
    krx = pd.read_csv('./src/krx_code.csv')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')

                date_list.append(root[1].split()[0].replace('.','-'))
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                date_list.append('error')
                comment_list.append('error')
                view_list.append(0)
                good_list.append(0)
                bad_list.append(0)   
        print(f'\r{i}페이지 크롤링 완료.',end='')   
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    df.to_csv(f"./src/네이버종토방댓글_{symbol}_{page}.csv", index=False)
    return df

In [5]:
df = get_comment_csv('카카오',500)
df

500페이지 크롤링 완료.

,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-26,외국인 기관 프로그램 매수다,6,0,0
1,2022-05-26,이게 이제야나오네..,19,3,0
2,2022-05-26,자회사 관계사 또 분할상장해서 5만원까지...,172,1,0
3,2022-05-26,카카오에서,99,0,0
4,2022-05-26,앵두형님이 바닥이란다,256,3,2
...,...,...,...,...,...
9995,2022-03-24,눌림목 모리나,257,0,0
9996,2022-03-24,윤석열이 외국인건보료 손본다,526,13,5
9997,2022-03-25,답글:윤석열이 외국인건보료 손본다,120,0,0
9998,2022-03-24,게카오 9900원에 아가리 쩌어어어어너어...,478,2,11
